In [1]:
import os
import json
from collections import Counter
import pandas as pd
from Bio import SeqIO

dataDir = "Data"
snpCompareDir = "SNPs_comparison"

In [2]:
if not os.path.exists(snpCompareDir):
    os.mkdir(snpCompareDir)

In [3]:
original = pd.read_csv(os.path.join(dataDir, "original.csv"))

In [4]:
minLen = 28000
maxN = 100

seqs = {}
outSeqs = []

for record in SeqIO.parse(os.path.join(dataDir, "sequences.fasta"), "fasta"):
    seqSum = Counter(str(record.seq).upper())
    if len(record) > minLen and seqSum['N'] < maxN:
        seqs[record.id] = record
        virusHost = original.loc[original["Accession.ID"] == record.id, "Host"]
        if virusHost.values[0] == "Manis javanica" or record.id == "EPI_ISL_402131":
            outSeqs.append(record)

SeqIO.write(outSeqs, os.path.join(snpCompareDir, "wild.fasta"), "fasta")

7

In [5]:
with open(os.path.join(dataDir, "genotypes.json")) as f:
    genotypes = json.load(f)

In [6]:
for gp in genotypes:
    outSeqs = []
    for ac in genotypes[gp]:
        record = seqs[ac]
        virusHost = original.loc[original["Accession.ID"] == record.id, "Host"]
        outSeqs.append(record)
    SeqIO.write(outSeqs, os.path.join(snpCompareDir, gp + ".fasta"), "fasta")